# Cortexハンズオン Part.2 
## Cortex Search サービス作成

次に、Snowflake内にあるドキュメントデータを検索することを可能にするCortex Searchサービスを作成します。

In [ ]:
-- 1. 使用するデータベースとスキーマを指定
USE DATABASE SNOWRETAIL_DB;
USE SCHEMA SNOWRETAIL_SCHEMA;

-- 2. Cortex Search Serviceを作成
CREATE OR REPLACE CORTEX SEARCH SERVICE snow_retail_search_service
    ON content                        -- 検索対象のテキストデータが含まれるカラム
    ATTRIBUTES title, document_type, department -- 検索結果に含めたい属性カラム
    WAREHOUSE = 'COMPUTE_WH' -- サービスのインデックス作成に使用するウェアハウス
    TARGET_LAG = '1 day'              -- データソーステーブルの変更がインデックスに反映されるまでの最大遅延時間
    EMBEDDING_MODEL = 'voyage-multilingual-2' -- テキストのベクトル化に使用する埋め込みモデル 
    AS
        -- 検索対象とするデータを定義するSELECT文
        SELECT
            document_id,  -- 各ドキュメントの一意なID
            title,        -- 属性として使用するタイトル
            content,      -- 検索対象のテキストコンテンツ
            document_type,-- 属性として使用するドキュメントタイプ
            department,   -- 属性として使用する部署名
            created_at,   -- オプション：メタデータとして含める作成日時
            updated_at,   -- オプション：メタデータとして含める更新日時
            version       -- オプション：メタデータとして含めるバージョン
        FROM SNOW_RETAIL_DOCUMENTS; -- データソースとなるテーブル